In [ ]:
import cv2
from ultralytics import YOLO

# --- Muat Model Deteksi Gangguan ---
model = YOLO('models/disruption-best-v3.pt')  # Pastikan model ini telah dilatih untuk gangguan di kelas
class_names = model.names       # Ambil label seperti 'disruption', 'noteTaking', dll

print(model.names)

# --- Buka Kamera ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Tidak bisa membuka kamera.")
    exit()

while True:
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        break

    # Balik gambar agar seperti cermin (opsional)
    frame = cv2.flip(frame, 1)

    # Deteksi menggunakan model
    results = model(frame, verbose=False)
    boxes = results[0].boxes

    # Anotasi hasil deteksi ke frame
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        label = f"{class_names[cls_id]}: {conf:.2%}"

        # Gambar kotak dan label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Tampilkan frame
    cv2.imshow("Disruption Detection", frame)

    # Tekan ESC (27) untuk keluar
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Bersihkan sumber daya
cap.release()
cv2.destroyAllWindows()


{0: 'Eating', 1: 'Fighting', 2: 'Sleeping', 3: 'Talking', 4: 'UOP', 5: 'phone', 6: 'sleep', 7: 'study', 8: 'hand-raising', 9: 'leaning over the table', 10: 'reading', 11: 'using phone', 12: 'writing'}
